In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io
from google.colab import files
from sklearn.model_selection import train_test_split
import warnings; warnings.simplefilter('ignore')
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def import_dataset():
  for _ in range(4):
    uploaded = files.upload()
  income_data = pd.read_csv('income_data.csv')
  user_data = pd.read_csv('user_data.csv')
  task_data = pd.read_csv('task_data.csv')
  assignment_data = pd.read_csv('assignment_data.csv')
  return income_data, user_data, task_data, assignment_data

In [ ]:
income_data, user_data, task_data, assignment_data = import_dataset()

Saving user_data.csv to user_data (2).csv


Saving task_data.csv to task_data (2).csv


Saving income_data.csv to income_data (2).csv


Saving assignment_data.csv to assignment_data (1).csv


In [ ]:
task_data.columns

Index(['Unnamed: 0', 'project_id', 'pool_id', 'title', 'description',
       'pool_startedAt', 'hasInstructions', 'mayContainAdultContent',
       'requesterID', 'requesterTrusted', 'lang', 'grade',
       'averageAcceptanceTimeSec', 'moneyAvgHourly', 'moneyMed', 'moneyTop10',
       'moneyMax3', 'reward'],
      dtype='object')

In [ ]:
task_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628 entries, 0 to 627
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                628 non-null    int64  
 1   project_id                628 non-null    int64  
 2   pool_id                   628 non-null    int64  
 3   title                     628 non-null    object 
 4   description               561 non-null    object 
 5   pool_startedAt            628 non-null    object 
 6   hasInstructions           628 non-null    int64  
 7   mayContainAdultContent    628 non-null    int64  
 8   requesterID               628 non-null    object 
 9   requesterTrusted          628 non-null    int64  
 10  lang                      88 non-null     object 
 11  grade                     473 non-null    float64
 12  averageAcceptanceTimeSec  130 non-null    float64
 13  moneyAvgHourly            585 non-null    float64
 14  moneyMed  

In [ ]:
task_data.shape

(628, 18)

In [ ]:
user_data.columns

Index(['Unnamed: 0', 'uid', 'country', 'language', 'birthdate', 'joined'], dtype='object')

In [ ]:
user_data.shape

(21, 6)

In [ ]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  21 non-null     int64 
 1   uid         21 non-null     int64 
 2   country     21 non-null     object
 3   language    21 non-null     object
 4   birthdate   21 non-null     object
 5   joined      21 non-null     object
dtypes: int64(2), object(4)
memory usage: 1.1+ KB


In [ ]:
income_data.columns

Index(['Unnamed: 0', 'project_id', 'uid', 'date', 'income'], dtype='object')

In [ ]:
income_data.shape

(1335, 5)

In [ ]:
income_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1335 entries, 0 to 1334
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1335 non-null   int64  
 1   project_id  1335 non-null   int64  
 2   uid         1335 non-null   int64  
 3   date        1335 non-null   object 
 4   income      1335 non-null   float64
dtypes: float64(1), int64(3), object(1)
memory usage: 52.3+ KB


In [ ]:
assignment_data.columns

Index(['Unnamed: 0', 'uid', 'project_id'], dtype='object')

In [ ]:
assignment_data.shape

(1053, 3)

In [ ]:
assignment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053 entries, 0 to 1052
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  1053 non-null   int64
 1   uid         1053 non-null   int64
 2   project_id  1053 non-null   int64
dtypes: int64(3)
memory usage: 24.8 KB


In [ ]:
log_data.columns

Index(['Unnamed: 0', 'id', 'uid', 'event', 'timestamp', 'url', 'tab_id',
       'scroll_count', 'blur_count', 'focus_count', 'click_count',
       'keypress_count'],
      dtype='object')

In [ ]:
log_data.shape

(152945, 12)

In [ ]:
log_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152945 entries, 0 to 152944
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      152945 non-null  int64  
 1   id              152945 non-null  int64  
 2   uid             152945 non-null  int64  
 3   event           152945 non-null  object 
 4   timestamp       152945 non-null  int64  
 5   url             125685 non-null  object 
 6   tab_id          152945 non-null  int64  
 7   scroll_count    60054 non-null   float64
 8   blur_count      60054 non-null   float64
 9   focus_count     60054 non-null   float64
 10  click_count     60054 non-null   float64
 11  keypress_count  60054 non-null   float64
dtypes: float64(5), int64(5), object(2)
memory usage: 14.0+ MB


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Clean the text data in task_data
task_data['description'] = task_data['description'].fillna('')
task_data['title'] = task_data['title'].fillna('')
task_data['text'] = task_data['title'] + ' ' + task_data['description']
task_data['text'] = task_data['text'].str.lower().str.replace('[^a-z0-9 ]+', '')

# Convert categorical features to numeric
task_data['requesterID'] = task_data['requesterID'].astype('category').cat.codes

# Compute the TF-IDF matrix
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(task_data['text'])

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Create a dictionary to map task IDs to their indices in the task_data DataFrame
id_to_index = dict(zip(task_data['project_id'], range(len(task_data))))

# Define the function to get top-K recommendations for a user
def get_top_k_recommendations(user_id, k=5):
    # Get the project IDs completed by the user
    completed_projects = set(income_data[income_data['uid'] == user_id]['project_id'])
    
    # Get the project IDs assigned to the user
    assigned_projects = set(assignment_data[assignment_data['uid'] == user_id]['project_id'])
    
    # Combine the completed and assigned project IDs
    selected_projects = completed_projects.union(assigned_projects)
    
    # Get the indices of the selected projects
    selected_indices = [id_to_index[project_id] for project_id in selected_projects if project_id in id_to_index]
    
    # Compute the similarity scores between the selected projects and all the other projects
    scores = cosine_sim[selected_indices].sum(axis=0)
    
    # Sort the scores and get the top-K recommendations
    top_k = np.argsort(-scores)[:k]
    
    # Get the project IDs corresponding to the top-K indices
    top_k_projects = [task_data.iloc[index]['project_id'] for index in top_k]

    # Get the titles of the projects
    titles_k =  [task_data.iloc[index]['title'] for index in top_k]
    
    return top_k_projects, titles_k

def ndcg_at_k(recommended_items, relevant_items, k):
    dcg = 0.0
    idcg = 0.0
    if len(recommended_items) < k:
        k = len(recommended_items)
    for i in range(k):
        item = recommended_items[i]
        if item in relevant_items:
            dcg += 1.0 / np.log2(i + 2)
    for i in range(min(len(relevant_items), k)):
        idcg += 1.0 / np.log2(i + 2)
    return dcg / idcg



def evaluate_model(k=5):
    # Initialize the scores
    map_k = 0.0
    ndcg_k = 0.0
    num_users = 0
    
    # Loop over all the users
    for user_id in user_data['uid']:
        # Get the ground truth and the predicted top-K recommendations
        project_ids = set(income_data[income_data['uid'] == user_id]['project_id'])
        # Filter the task data to include only the tasks for the specified projects
        project_tasks = task_data[task_data['project_id'].isin(project_ids)]
        # Get a list of the titles for the project tasks
        ground_truth = set(list(project_tasks['title']))
        p_ids, titles_k = get_top_k_recommendations(user_id, k=k)
        predicted = set(titles_k)
        # predicted = set(get_top_k_recommendations(user_id, k=k))
        
        # Compute the MAP@K and NDCG@K for the user
        num_hits = len(ground_truth.intersection(predicted))
        if num_hits > 0:
            precision = float(num_hits) / float(k)
            recall = float(num_hits) / float(len(ground_truth))
            map_k += precision
            ndcg_k += ndcg_at_k(list(predicted), list(ground_truth), k=k)
            num_users += 1
            
    # Normalize the scores by the number of users
    map_k /= float(num_users)
    ndcg_k /= float(num_users)
    
    return map_k, ndcg_k

In [ ]:
evaluate_model()

{'Get snapshot of all coupons', 'Относится ли изображение к категории?', 'Take 10 photos of your face', "Does coupon screenshot match the provided coupon's information? (USA)", 'Does the provided link contain coupons? (South Africa)', 'Is the PRODUCT suitable for the INTEREST?', 'Validation of English Ad Description', 'Does the image belong to the category?', 'Get coupon information from websites (Canada)', 'Product Inventory Validation V1', 'Validation of English Ad Title', 'Is there a short answer in the text?', 'Is the given information match the screenshot 2', 'Generic Scenario Testing Extension (English: United States)', 'MMWatermarkImageDetection', 'Creation of English Ad Description', 'Image Relevance Superfresh V2 SFMMIR - English', 'Product Price Validation V1', 'Compare Search Results - English - V2', 'Should the sentence be translated? [en]', 'Is the ad relevant to the query', 'Оценка правильности категории', 'Extract offer details ', 'Check if two audios has similar style',

(0.47999999999999987, 0.6713193066525202)

In [ ]:
len(user_data)

21

The values of map_k and ndcg_k are evaluation metrics that provide an estimate of how well your recommendation model is performing.

map_k (mean average precision) is a metric that takes into account both the relevance and order of the recommended items. The value of map_k ranges from 0 to 1, where a value of 1 indicates that all recommended items were relevant and ordered perfectly. In this case, a value of 0.4 indicates that the model achieved a moderate level of precision in the top k recommended items.

ndcg_k (normalized discounted cumulative gain) is another metric that evaluates the quality of the recommendations, taking into account the position of the relevant items in the recommendation list. The value of ndcg_k ranges from 0 to 1, where a value of 1 indicates that all the relevant items were at the top of the recommendation list. In this case, a value of 0.58 indicates that the model achieved a moderate level of performance in ranking relevant items at the top of the recommendation list.

In [ ]:
from tabulate import tabulate

def display_recommendations(k):

    for i in user_data['uid'].unique():

      list1, list2 = get_top_k_recommendations(i, k)
      # Zip the two lists to create a list of tuples
      zipped_lists = list(zip(list1, list2))

      print("User ID: ", i)
      # Display the zipped lists as a table
      print(tabulate(zipped_lists, headers=['Porject ID', 'Title']))

      print('--------------------------------')

In [ ]:
display_recommendations(5)

User ID:  1316040758
  Porject ID  Title
------------  ---------------------------------------------
       84962  Get coupon information from websites (USA)
       84962  Get coupon information from websites (USA)
       91289  Get coupon information from websites (Canada)
       91298  Get coupon information from websites (India)
       91298  Get coupon information from websites (India)
--------------------------------
User ID:  1429235843
  Porject ID  Title
------------  -------------------------------------
      107309  Is the product relevant to the query?
      107309  Is the product relevant to the query?
       74976  MMRepresentativeImageV2
       54170  Microsoft Bing App test
       54170  Microsoft Bing App test
--------------------------------
User ID:  1430094402
  Porject ID  Title
------------  -----------------------------------------------
      113582  Сравните Результаты Поиска – Русский язык - V2-
       89350  Crowd Spam Labeling - Russian
      113574  Compare